# This is the code for the best classification model we've trained:

*   LSTM mdoel with one hidden layer
*   Three inputs: price, sentiment, percentage (price variation after 24 hours) 
*   We use TensorBoard to track and visualize the performance metrics such as loss and accuracy. We've commented out the codes for connecting with TensorBoard because they only work in **Google Colab** environment. If you are using **Jupyter**, here is the link for setting up TensorBoard: https://www.tensorflow.org/tensorboard/r2/tensorboard_in_notebooks





 


In [3]:
import pandas as pd
import numpy as np
import keras
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

# The following block of codes are for using TensorBoard in Google Colab

In [0]:
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
#py.init_notebook_mode(connected=True)
#tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://1861ff8f.ngrok.io


# Preprocessing data input

*   The original data file is called "24hour_price_emotion_4label.csv". pLease change the file path accordingly while reading the file.
*   List item




**Read the data file "24hour_price_emotion_4label.csv" :**

In [4]:
data =  pd.read_csv ('../Files/24hour_price_emotion_4label.csv')
data['Date']=pd.to_datetime(data['Date'])
data = data.sort_values(['Date'], ascending=[True])
data.head()

,Unnamed: 0,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Average_Anger,...,Average_Sadness,Average_Fear,Average_Disgust,Maximum_Anger,Maximum_Joy,Maximum_Sadness,Maximum_Fear,Maximum_Disgust,Percentage,label
0,31835,1.444309e+09,2015-10-08 13:00:00,BTCUSD,0.00,245.00,0.00,245.00,0.606654,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004082,2
1,31834,1.444313e+09,2015-10-08 14:00:00,BTCUSD,245.00,245.00,244.50,245.00,4.453649,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004082,2
2,31833,1.444316e+09,2015-10-08 15:00:00,BTCUSD,245.00,245.00,244.92,244.92,3.016926,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003756,2
3,31832,1.444320e+09,2015-10-08 16:00:00,BTCUSD,244.92,244.92,244.25,244.25,3.895252,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001024,2
4,31831,1.444324e+09,2015-10-08 17:00:00,BTCUSD,244.25,244.99,244.02,244.99,3.920632,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005266,2


**Transform the data using MInMax Scaler :**

In [6]:
datalength = data.shape[0]

sentiment = data[['Average_Anger', 'Average_Joy', 'Average_Sadness', 'Average_Fear', 'Average_Disgust']].copy()
price = data[['Close']].copy()

scaler_value = MinMaxScaler(feature_range=(0, 1))
values = data['Close'].values.reshape(datalength,1)
values = values.astype('float32')
values = scaler_value.fit_transform(values)
values = values.reshape(datalength,1,1)

scaler_sentiment = MinMaxScaler(feature_range=(0,1))
sentiments = sentiment.values
sentiments = scaler_sentiment.fit_transform(sentiments)
sentiments = sentiments.reshape(datalength,1,5)

scaler_percentage = MinMaxScaler(feature_range=(0, 1))
percentage = data['Percentage'].values.reshape(datalength,1)
percentage = scaler_percentage.fit_transform(percentage)
percentage = percentage.reshape(datalength,1,1)

**Split the data into training set and test set :**

In [7]:
train_size = int(len(values) * 0.7)
test_size = len(values) - train_size

trainP = values[0:train_size,:,:]
testP = values[train_size:len(values),:,:]

trainS = sentiments[0:train_size,:,:]
testS = sentiments[train_size:len(values),:,:]

trainpercentage = percentage[0:train_size,:,:]
testpercentage= percentage[train_size:len(values),:,:]


**Create three matrix (price, sentiment, percentage) as the inputs for the LSTM model :**


*   define three functions to process the dataset
*  we set the look_back (lag) to 51



In [8]:
def create_dataset_price(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        a = dataset[i-look_back : i, 0]
        dataX.append(a)
    return np.array(dataX)
  
def create_dataset_sentiment(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        a = dataset[i-look_back : i, 0]
        dataX.append(a)
    return np.array(dataX)
  
def create_dataset_percentage(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        a = dataset[i-look_back : i, 0]
        dataX.append(a)
    return np.array(dataX) 

look_back = 51
  
trainPrice = create_dataset_price(trainP,look_back)
testPrice = create_dataset_price(testP,look_back)
trainSentiment = create_dataset_sentiment(trainS,look_back)
testSentiment = create_dataset_sentiment(testS,look_back)
trainPercentage = create_dataset_percentage(trainpercentage,look_back)
testPercentage = create_dataset_percentage(testpercentage,look_back)

trainPrice = np.reshape(trainPrice, (trainPrice.shape[0], 1, trainPrice.shape[1]))
testPrice = np.reshape(testPrice, (testPrice.shape[0], 1, testPrice.shape[1]))
trainSentiment = np.reshape(trainSentiment, (trainSentiment.shape[0], 51, 5))
testSentiment = np.reshape(testSentiment, (testSentiment.shape[0],51, 5))  
trainPercentage = np.reshape(trainPercentage, (trainPercentage.shape[0], 1, trainPercentage.shape[1]))
testPercentage = np.reshape(testPercentage, (testPercentage.shape[0], 1, testPercentage.shape[1]))

**preprocess the label to categorical labels :**

In [9]:
  from keras.utils import to_categorical
  
  label = data['label'].values  
  label = to_categorical(label)
  trainLabel = label[look_back : trainPrice.shape[0] + look_back] 
  testLabel = label[trainPrice.shape[0] + 2*look_back  : label.shape[0]] 

# Build the LSTM model for classification :


*   Three independent input (price, sentiment, percentage)
*   One hidden layer for each input
*   In the Dense layer, we use 'softmax' as the activation function
*   We use 'adam' as the optimizer, 'categorical_crossentropy' as the loss function, 'accuracy' as the model metrics





In [10]:
def buildModel(lookback):
  
    price = Input(shape = (1, lookback),name='price')
    priceLayers = LSTM(64, return_sequences=False)(price)
    
    sentiment = Input(shape=(51, 5),name='sentiment')
    sentimentLayers = LSTM(64, return_sequences=False)(sentiment)
    
    percentage = Input(shape=(1, lookback),name='percentage')
    percentageLayers = LSTM(64, return_sequences=False)(percentage)
        
    output = keras.layers.concatenate([priceLayers,
                                       sentimentLayers,
                                       percentageLayers])
    
    output2 = Dense(4, activation='softmax',name='output')(output)
    
    model = Model(inputs=[price,sentiment,percentage], outputs=[output2])
   
    model.compile(optimizer = 'adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Train the model

In [11]:
look_back = 51  
lstm = buildModel(look_back)
lstm.fit([trainPrice,trainSentiment,trainPercentage],trainLabel,
              epochs = 50, 
              batch_size = 64, 
              verbose = 1,
              validation_split=0.1,
              # callbacks=[TensorBoardColabCallback(tbc)], 
              shuffle=True)

W0909 15:20:47.665934 15632 deprecation_wrapper.py:119] From C:\Users\Ray\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 15:20:47.701838 15632 deprecation_wrapper.py:119] From C:\Users\Ray\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0909 15:20:47.703835 15632 deprecation_wrapper.py:119] From C:\Users\Ray\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0909 15:20:48.183433 15632 deprecation_wrapper.py:119] From C:\Users\Ray\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0909 15:20:48.204123 15632 deprecation_wrapper.py:119] From C:\Users\Ray\Anaconda3\lib\

Train on 19995 samples, validate on 2222 samples
Epoch 1/50
19995/19995 [==============================] - 11s 533us/step - loss: 1.1962 - acc: 0.4612 - val_loss: 0.9240 - val_acc: 0.6656
Epoch 2/50
19995/19995 [==============================] - 10s 488us/step - loss: 0.9990 - acc: 0.5854 - val_loss: 0.7968 - val_acc: 0.6971
Epoch 3/50
19995/19995 [==============================] - 12s 575us/step - loss: 0.9058 - acc: 0.6279 - val_loss: 0.7303 - val_acc: 0.7106
Epoch 4/50
19995/19995 [==============================] - 11s 571us/step - loss: 0.8539 - acc: 0.6535 - val_loss: 0.6977 - val_acc: 0.7219
Epoch 5/50
19995/19995 [==============================] - 11s 574us/step - loss: 0.8154 - acc: 0.6736 - val_loss: 0.6667 - val_acc: 0.7354
Epoch 6/50
19995/19995 [==============================] - 12s 582us/step - loss: 0.7854 - acc: 0.6925 - val_loss: 0.6668 - val_acc: 0.7340
Epoch 7/50
19995/19995 [==============================] - 13s 626us/step - loss: 0.7671 - acc: 0.6983 - val_loss: 0.6

**Model summary :**

In [12]:
lstm.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
price (InputLayer)              (None, 1, 51)        0                                            
__________________________________________________________________________________________________
sentiment (InputLayer)          (None, 51, 5)        0                                            
__________________________________________________________________________________________________
percentage (InputLayer)         (None, 1, 51)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 64)           29696       price[0][0]                      
____________________________________________________________________________________________

# Evaluate the model with test set data we left unseen


*   **class 0**:   price variation after 24 hours > 1.5%
*   **class 1**:   0 <= price variation after 24 hours <= 1.5%
*   **class 2**:   -1.5% <= price variation after 24 hours < 0
*   **class 3**:   price variation after 24 hours < -1.5%





In [13]:
accr = lstm.evaluate([testPrice, testSentiment,testPercentage],testLabel)
print('Test set Loss: {:0.3f}  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print('\n')

labels = [0,1,2,3]; predict_label =[]; true_label=[];

pred = lstm.predict([testPrice,testSentiment,testPercentage])

for i in range(testPrice.shape[0]):
  predict_label.append(labels[np.argmax(pred[i])]) 
  true_label.append(labels[np.argmax(testLabel[i])])
  
count0=0; count0_correct=0
count1=0; count1_correct=0
count2=0; count2_correct=0
count3=0; count3_correct=0

for i in range(len(predict_label)):
  if (predict_label[i] ==0):
    count0 =count0+1
    if (predict_label[i] == true_label[i]):
      count0_correct = count0_correct+1
  elif (predict_label[i] ==1):
    count1 = count1+1
    if (predict_label[i] == true_label[i]):
      count1_correct = count1_correct+1
  elif (predict_label[i] ==2):
    count2 = count2+1
    if (predict_label[i] == true_label[i]):
      count2_correct = count2_correct+1
  elif (predict_label[i] ==3):
    count3 = count3+1
    if (predict_label[i] == true_label[i]):
      count3_correct = count3_correct+1
      
print(count0)
print(count1)
print(count2)
print(count3)
print()
print(count0_correct)
print(count1_correct)
print(count2_correct)
print(count3_correct)
print()
print('accuracy for class 0: ', count0_correct/count0)
print('accuracy for class 1: ', count1_correct/count1)
print('accuracy for class 2: ', count2_correct/count2)
print('accuracy for class 3: ', count3_correct/count3)

9493/9493 [==============================] - 2s 162us/step
Test set Loss: 0.626  Accuracy: 0.769


2205
2270
2395
2623

1911
1728
1710
1955

accuracy for class 0:  0.8666666666666667
accuracy for class 1:  0.7612334801762115
accuracy for class 2:  0.7139874739039666
accuracy for class 3:  0.7453297750667175
